In [ ]:
from data import *
from utils import *
import pandas as pd
import pygwalker as pyg

import neurokit2 as nk

train_data = np.load('/workspace/signal/PVC-NET_/dataset//mit-bih-arrhythmia-database-1.0.0_train_20230216.npy',allow_pickle=True) # B x (C) x Signal
valid_data = np.load('/workspace/signal/PVC-NET_/dataset/mit-bih-arrhythmia-database-1.0.0_valid_20230216.npy',allow_pickle=True) # B x (C) x Signal
test_data = np.load('/workspace/signal/PVC-NET_/dataset/mit-bih-arrhythmia-database-1.0.0_test_20230216.npy',allow_pickle=True) # B x (C) x Signal

AMC_data  = np.load('/workspace/signal/PVC-NET_/dataset/AMC_PeakLabel_3rd_125Hz.npy',allow_pickle=True) # 497 samples
CPSC2020_data  = np.load('/workspace/signal/PVC-NET_/dataset/CPSC2020_testSeg_ver2.npy',allow_pickle=True)
CU_data  = np.load('/workspace/signal/PVC-NET_/dataset/cu-ventricular-tachyarrhythmia-database-1.0.0_testSeg.npy',allow_pickle=True)
ESC_data  = np.load('/workspace/signal/PVC-NET_/dataset/european-st-t-database-1.0.0_testSeg.npy',allow_pickle=True)
FANTASIA_data = np.load('/workspace/signal/PVC-NET_/dataset/fantasia-database-1.0.0_testSeg.npy', allow_pickle=True) # B x (C) x Signal
INCART_data  = np.load('/workspace/signal/PVC-NET_/dataset/INCART_testSeg.npy',allow_pickle=True)
NS_data = np.load('/workspace/signal/PVC-NET_/dataset/mit-bih-noise-stress-test-database-1.0.0_testSeg.npy',allow_pickle=True)
STDB_data = np.load('/workspace/signal/PVC-NET_/dataset/mit-bih-st-change-database-1.0.0_testSeg.npy',allow_pickle=True)
SVDB_data = np.load('/workspace/signal/PVC-NET_/dataset/mit-bih-supraventricular-arrhythmia-database-1.0.0_testSeg.npy',allow_pickle=True)

In [ ]:
df = pd.read_csv(, parse_dates=['date'])
gwalker = pyg.walk(df)

In [ ]:
trainvalid_data = np.concatenate((train_data, valid_data))
np.save('mit-bih-arrhythmia-database-1.0.0_trainvalid_2023.npy',)

In [ ]:
len(train_data), len(valid_data), len(test_data), len(trainvalid_data)

In [ ]:
train_data = np.concatenate((train_data,STDB_data))
len(train_data)

In [ ]:
# data = FANTASIA_data
data = CU_data

print(len(data))
for i in range(len(data)):
    if len(data[i]['symbol_PVC'])!=0:
        print(i,data[i]['symbol_PVC'])

In [ ]:
!pip install monai==1.0.0

In [ ]:
import monai
monai.__version__

In [ ]:
import torch
import models
from models.resnet import *
from models.efficientnet import *
from models.ffc import *

In [ ]:
# net = resnet34(n_input_channels=1, spatial_dims=1, module='acm')
# net = resnet34(n_input_channels=1, spatial_dims=1, module='cbam')
# net = resnet34(n_input_channels=1, spatial_dims=1, module='deeprft')
# net = resnet34(n_input_channels=1, spatial_dims=1, module='ffc')
# net = resnet34(n_input_channels=1, spatial_dims=1, module='mha')
# net = resnet34(n_input_channels=1, spatial_dims=1, module='nlnn')
# net = resnet34(n_input_channels=1, spatial_dims=1, module='se')
net = resnet34(n_input_channels=1, spatial_dims=1, module='SCM9')

In [ ]:
# net = EfficientNetBNFeatures(model_name = 'efficientnet-b0', spatial_dims=1,module='acm')
# net = EfficientNetBNFeatures(model_name = 'efficientnet-b0', spatial_dims=1,module='cbam')
# net = EfficientNetBNFeatures(model_name = 'efficientnet-b0', spatial_dims=1,module='deeprft')
# net = EfficientNetBNFeatures(in_channels=1, model_name = 'efficientnet-b0', spatial_dims=1, module='ffc')
# net = EfficientNetBNFeatures(model_name = 'efficientnet-b0', spatial_dims=1,module='nlnn')
net = EfficientNetBNFeatures(in_channels=1, model_name = 'efficientnet-b0', spatial_dims=1, module='SCM')


In [ ]:
x = torch.rand(1,1,1280)
yhat = net(x)

In [ ]:
for yh in yhat:
    print(yh.shape)

In [ ]:
net

In [ ]:
# def count_twos(n):
#     count = 0
#     while n % 2 == 0:
#         count += 1
#         n = n // 2

#     if count >= 3:
#         return 2**3    
#     else:
#         return 0

# a = count_twos(32)
# a